In [1]:
import duckdb
import yaml
import datetime
import openai

In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model=PARAM['vector_embedding_model']):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3]:
con = duckdb.connect('md:')

con.execute("install duckpgq from community;")
con.execute("load duckpgq;")

con.execute("install ducklake;")
con.execute("load ducklake;")

con.execute("install postgres;")
con.execute("load postgres;")

con.sql(f"""
USE ducklake_omop;
""")


con.sql("DROP PROPERTY GRAPH IF EXISTS drug_graph")

con.sql("""
CREATE PROPERTY GRAPH drug_graph
  VERTEX TABLES (
    Drug, Disease, MOA
  )
EDGE TABLES (
  Drug_Disease 	SOURCE KEY ('from') REFERENCES Drug (cui)
                DESTINATION KEY ('to') REFERENCES Disease (cui)
  LABEL MAY_TREAT,
  Drug_MOA SOURCE KEY ('from') REFERENCES Drug (cui)
          DESTINATION KEY ('to') REFERENCES MOA (id)
  LABEL HAS_MOA
);
""")

Attempting to automatically open the SSO authorization page in your default browser.
Please open this link to login into your account: https://auth.motherduck.com/activate?user_code=WCTG-BZXS


Token successfully retrieved ✅

You can display the token and store it as an environment variable to avoid having to log in again:
  PRAGMA PRINT_MD_TOKEN;


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [4]:
con.sql("""SELECT DISTINCT disease_name
  FROM GRAPH_TABLE (drug_graph
  MATCH
  (mo:MOA WHERE mo.name = 'GABA B Receptor Interactions')<-[h:HAS_MOA]-(i:Drug)-[m:MAY_TREAT]->(c:Disease)
  COLUMNS (c.name AS disease_name)
  )
  LIMIT 5;
""")

┌─────────────────────┐
│    disease_name     │
│       varchar       │
├─────────────────────┤
│ Alzheimer's Disease │
│ Bipolar Disorder    │
│ Mood Disorders      │
│ Absence Epilepsy    │
│ Epilepsy            │
└─────────────────────┘

In [5]:
con.sql(f"""
ATTACH 'host=192.168.1.29 port=5432 user={PARAM["omop_postgres_user"]} password={PARAM["omop_postgres_password"]} dbname=postgres' AS broadsea_db (TYPE POSTGRES);
""")

In [6]:
con.sql("""
SELECT DISTINCT ce.person_id
FROM ducklake_omop.Trial t,
     UNNEST(t.condition_cui) AS u(cui)
JOIN broadsea_db.demo_cdm.source_to_concept_map scm
  ON scm.source_code = u.cui
JOIN broadsea_db.demo_cdm.condition_era ce
  ON ce.condition_concept_id = scm.target_concept_id
WHERE t."Posting ID" = 108
LIMIT 10;
""")

┌───────────┐
│ person_id │
│   int32   │
├───────────┤
│       547 │
│       508 │
│       509 │
│       626 │
│       670 │
│       650 │
│       764 │
│       273 │
│       561 │
│       166 │
├───────────┤
│  10 rows  │
└───────────┘

In [7]:
con.sql("""
SELECT DISTINCT de.person_id
FROM ducklake_omop.Trial t,
     UNNEST(t.CUI_umls) AS u(cui)
JOIN broadsea_db.demo_cdm.source_to_concept_map scm
  ON scm.source_code = u.cui
JOIN broadsea_db.demo_cdm.drug_era de
  ON de.drug_concept_id = scm.target_concept_id
WHERE t."Posting ID" = 121
LIMIT 10;
""")

┌───────────┐
│ person_id │
│   int32   │
├───────────┤
│        35 │
│       194 │
│       255 │
│       323 │
│       379 │
│       457 │
│       496 │
│       512 │
│       676 │
│       710 │
├───────────┤
│  10 rows  │
└───────────┘

In [8]:
con.sql("""SELECT DISTINCT drug_cui
  FROM GRAPH_TABLE (drug_graph
  MATCH
  (i:Drug)-[m:MAY_TREAT]->(c:Disease WHERE c.name = 'Atrial Flutter')
  COLUMNS (i.cui AS drug_cui)
  )
  LIMIT 30;
""")

┌──────────┐
│ drug_cui │
│ varchar  │
├──────────┤
│ C0082966 │
│ C0034414 │
│ C0012702 │
│ C0700548 │
│ C0037707 │
│ C0025859 │
│ C0114771 │
│ C0033216 │
│ C0012258 │
│ C0700468 │
│ C0012373 │
│ C0004147 │
│ C0011689 │
│ C0012265 │
│ C0042523 │
│ C0700579 │
├──────────┤
│ 16 rows  │
└──────────┘

In [9]:
con.sql("""
SELECT DISTINCT de.person_id, drugs.drug_name
FROM (
  SELECT DISTINCT drug_cui, drug_name
  FROM GRAPH_TABLE (drug_graph
    MATCH
    (i:Drug)-[m:MAY_TREAT]->(c:Disease WHERE c.name = 'Atrial Flutter')
    COLUMNS (i.cui AS drug_cui, i.name AS drug_name)
  )
) AS drugs
JOIN broadsea_db.demo_cdm.source_to_concept_map scm
  ON scm.source_code = drugs.drug_cui
JOIN broadsea_db.demo_cdm.drug_era de
  ON de.drug_concept_id = scm.target_concept_id
ORDER BY drug_name DESC
""")

┌───────────┬───────────┐
│ person_id │ drug_name │
│   int32   │  varchar  │
├───────────┼───────────┤
│       859 │ verapamil │
│       450 │ verapamil │
│       562 │ verapamil │
│       999 │ verapamil │
│       455 │ verapamil │
│       529 │ verapamil │
│       843 │ verapamil │
│      1111 │ verapamil │
│       830 │ verapamil │
│       759 │ verapamil │
│        ·  │    ·      │
│        ·  │    ·      │
│        ·  │    ·      │
│       529 │ digoxin   │
│       222 │ digoxin   │
│       998 │ digoxin   │
│       308 │ digoxin   │
│       770 │ digoxin   │
│       310 │ digoxin   │
│       450 │ digoxin   │
│       540 │ digoxin   │
│       722 │ digoxin   │
│       740 │ digoxin   │
├───────────┴───────────┤
│ 80 rows     2 columns │
│ (20 shown)            │
└───────────────────────┘

In [10]:
con.sql("""
SELECT drug_name, person_id
FROM (
  SELECT drugs.drug_name,
         de.person_id,
         ROW_NUMBER() OVER (PARTITION BY drugs.drug_name ORDER BY de.person_id) AS rn
  FROM (
    SELECT DISTINCT drug_cui, drug_name
    FROM GRAPH_TABLE (drug_graph
      MATCH
      (i:Drug)-[m:MAY_TREAT]->(c:Disease WHERE c.name = 'Atrial Flutter')
      COLUMNS (i.cui AS drug_cui, i.name AS drug_name)
    )
  ) AS drugs
  JOIN broadsea_db.demo_cdm.source_to_concept_map scm
    ON scm.source_code = drugs.drug_cui
  JOIN broadsea_db.demo_cdm.drug_era de
    ON de.drug_concept_id = scm.target_concept_id
)
WHERE rn <= 5
ORDER BY drug_name;

""")

┌───────────┬───────────┐
│ drug_name │ person_id │
│  varchar  │   int32   │
├───────────┼───────────┤
│ digoxin   │         3 │
│ digoxin   │        33 │
│ digoxin   │        97 │
│ digoxin   │       111 │
│ digoxin   │       161 │
│ verapamil │         3 │
│ verapamil │        33 │
│ verapamil │        97 │
│ verapamil │       111 │
│ verapamil │       210 │
├───────────┴───────────┤
│ 10 rows     2 columns │
└───────────────────────┘

In [11]:
con.sql("""
SELECT drug_name, person_id, drug_concept_id
FROM (
  SELECT drugs.drug_name,
         de.person_id,
         de.drug_concept_id,
         ROW_NUMBER() OVER (PARTITION BY drugs.drug_name ORDER BY de.person_id) AS rn
  FROM (
    SELECT DISTINCT drug_cui, drug_name
    FROM GRAPH_TABLE (drug_graph
      MATCH
      (i:Drug)-[h:HAS_MOA]->(m:MOA WHERE m.name = 'Glucocorticoid Receptor Agonists')
      COLUMNS (i.cui AS drug_cui, i.name AS drug_name)
    )
  ) AS drugs
  JOIN broadsea_db.demo_cdm.source_to_concept_map scm
    ON scm.source_code = drugs.drug_cui
  JOIN broadsea_db.demo_cdm.drug_era de
    ON de.drug_concept_id = scm.target_concept_id
  WHERE de.drug_era_start_date > '2013-01-01'
)
WHERE rn <= 5
ORDER BY drug_name;

""")

┌────────────────┬───────────┬─────────────────┐
│   drug_name    │ person_id │ drug_concept_id │
│    varchar     │   int32   │      int32      │
├────────────────┼───────────┼─────────────────┤
│ fluticasone    │        88 │         1149380 │
│ fluticasone    │       256 │         1149380 │
│ fluticasone    │       369 │         1149380 │
│ fluticasone    │       388 │         1149380 │
│ fluticasone    │       457 │         1149380 │
│ hydrocortisone │       185 │          975125 │
│ hydrocortisone │       221 │          975125 │
│ hydrocortisone │       221 │          975125 │
│ hydrocortisone │       231 │          975125 │
│ hydrocortisone │       525 │          975125 │
│ prednisone     │         7 │         1551099 │
│ prednisone     │        35 │         1551099 │
│ prednisone     │        99 │         1551099 │
│ prednisone     │       185 │         1551099 │
│ prednisone     │       388 │         1551099 │
├────────────────┴───────────┴─────────────────┤
│ 15 rows           

In [12]:
con.sql("""
SELECT drug_name, person_id, drug_concept_id, drug_era_start_date, location_id
FROM (
  SELECT drugs.drug_name,
         de.person_id,
         de.drug_concept_id,
         pe.location_id,
         de.drug_era_start_date, -- <--- ADDED THIS LINE
         ROW_NUMBER() OVER (PARTITION BY drugs.drug_name ORDER BY de.person_id) AS rn
  FROM (
    SELECT DISTINCT drug_cui, drug_name
    FROM GRAPH_TABLE (drug_graph
      MATCH
      (i:Drug)-[h:HAS_MOA]->(m:MOA WHERE m.name = 'Glucocorticoid Receptor Agonists')
      COLUMNS (i.cui AS drug_cui, i.name AS drug_name)
    )
  ) AS drugs
  JOIN broadsea_db.demo_cdm.source_to_concept_map scm
    ON scm.source_code = drugs.drug_cui
  JOIN broadsea_db.demo_cdm.drug_era de
    ON de.drug_concept_id = scm.target_concept_id
  JOIN broadsea_db.demo_cdm.person pe
    ON de.person_id = pe.person_id
  WHERE de.drug_era_start_date > '2013-01-01' AND pe.location_id = 1
)
WHERE rn <= 5
ORDER BY drug_name;
""")

┌─────────────┬───────────┬─────────────────┬─────────────────────┬─────────────┐
│  drug_name  │ person_id │ drug_concept_id │ drug_era_start_date │ location_id │
│   varchar   │   int32   │      int32      │        date         │    int32    │
├─────────────┼───────────┼─────────────────┼─────────────────────┼─────────────┤
│ fluticasone │        88 │         1149380 │ 2015-07-21          │           1 │
│ fluticasone │       369 │         1149380 │ 2014-01-29          │           1 │
│ fluticasone │       388 │         1149380 │ 2015-06-19          │           1 │
│ prednisone  │       388 │         1551099 │ 2013-08-31          │           1 │
└─────────────┴───────────┴─────────────────┴─────────────────────┴─────────────┘

In [13]:
con.close()